In [ ]:
import requests
import csv

def get_access_token(client_id, client_secret):
    auth_url = "https://accounts.spotify.com/api/token"
    auth_response = requests.post(
        auth_url,
        {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
        },
    )
    if auth_response.status_code == 200:
        return auth_response.json().get("access_token")
    return None


def search_track(track_name,artist, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    tr=track_name.lower().replace("'", "")
    ar=artist.lower().replace("'", "")
    query = f"track:{tr} artist:{ar}"
    params = {"q": query, "type": "track", "limit": 1}
    search_url = "https://api.spotify.com/v1/search"
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200 and "tracks" in response.json():
        tracks = response.json()["tracks"]["items"]
        if tracks:
            track = tracks[0]
            album_id = track["album"]["id"]
            return track, album_id
    return None, None


def get_audio_features(track_id, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    response = requests.get(features_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    return None


def get_album_and_artist_genres(album_id, artist_id, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}

    album_url = f"https://api.spotify.com/v1/albums/{album_id}"
    response = requests.get(album_url, headers=headers)

    if response.status_code == 200 and "genres" in response.json():
        album_genres = response.json()["genres"]
        if album_genres:
            return album_genres

    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    response = requests.get(artist_url, headers=headers)

    if response.status_code == 200 and "genres" in response.json():
        artist_genres = response.json()["genres"]
        if artist_genres:
            return artist_genres

    return ["Unknown"]


def process_csv_and_write_to_csv(input_csv, output_csv, client_id, client_secret):
    with open(input_csv, newline="", encoding="utf-8") as csvfile, open(
        output_csv, "w", newline="", encoding="utf-8"
    ) as outcsv:
        reader = csv.reader(csvfile)
        fieldnames = [
            "artist",
            "song",
            "genre",
            "id",
            "release_year",
            "danceability",
            "energy",
            "key",
            "loudness",
            "mode",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "liveness",
            "valence",
            "tempo",
            "duration_ms",
            "time_signature",
        ]
        writer = csv.DictWriter(outcsv, fieldnames=fieldnames)
        writer.writeheader()
        access_token = get_access_token(client_id, client_secret)
        if not access_token:
            print("Failed to obtain Spotify access token")
            return
        next(reader)
        # Process each row in the input CSV
        for row in reader:
            song = row[0].strip()  # Song name
            artist = row[1].strip()  # Artist name

            track, album_id = search_track(song,artist, access_token)

            if track and album_id:
                # Get audio features of the track
                audio_features = get_audio_features(track["id"], access_token)

                # Get genres from album or artist
                artist_id = track["artists"][0]["id"]
                genres = get_album_and_artist_genres(album_id, artist_id, access_token)
                genre = genres

                # Gather track information and audio features
                track_details = {
                    "artist": artist,
                    "song": song,
                    "genre": genre,
                    "id": track["id"],
                    "release_year": track["album"]["release_date"][:4],
                    "danceability": audio_features.get("danceability", 0),
                    "energy": audio_features.get("energy", 0),
                    "key": audio_features.get("key", 0),
                    "loudness": audio_features.get("loudness", 0),
                    "mode": audio_features.get("mode", 0),
                    "speechiness": audio_features.get("speechiness", 0),
                    "acousticness": audio_features.get("acousticness", 0),
                    "instrumentalness": audio_features.get("instrumentalness", 0),
                    "liveness": audio_features.get("liveness", 0),
                    "valence": audio_features.get("valence", 0),
                    "tempo": audio_features.get("tempo", 0),
                    "duration_ms": audio_features.get("duration_ms", 0),
                    "time_signature": audio_features.get("time_signature", 0),
                }

                writer.writerow(track_details)
            else:
                print(f"No track found for {song} by {artist}")

input_csv = 'in/tables/nomineeswinners.csv' 
output_csv = 'out/tables/spotify.csv'

client_id = "***"
client_secret = "***"
process_csv_and_write_to_csv(input_csv, output_csv, client_id, client_secret)